In [34]:
#Version del scraper de Maldita 2.0
#Esta versión ha sido desarrollada para de una forma limpia poder scrapear maldita seleccionando un página en concreto.

In [35]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import pickle

In [36]:
pagina = 34

In [37]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces

In [38]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
    #print(title)

    date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="article-body")
    body2 = body.find_all("p", class_="")

    for e in body2:
        content_list.append(e.get_text().replace("\xa0", " ").replace("\\", ""))
    
    content = ''.join(content_list)
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url, "Fuente": "Maldita.es"}
    
    return article

In [39]:
lista_enlaces = []
suficiente = True

#Cargo las URLS.
print('Cargando URLs...')

link =  "https://maldita.es/malditobulo/page/" + str(pagina) + "/"
arrayurls= get_urls(link, lista_enlaces)
        
lista_articulos = []   

for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)
    
print(len(lista_enlaces))
print(lista_enlaces)

Cargando URLs...
24
['https://maldita.es/malditobulo/2020/02/28/bulo-chicos-negros-bebiendo-refrescos-supermercado-pagadores-pensiones-espana/', 'https://maldita.es/malditobulo/2020/02/28/diario-abc-dos-posibles-infectados-coronavirus-gines-benacazon-sevilla-andalucia/', 'https://maldita.es/malditobulo/2020/02/28/meme-ayuso-no-tocar-arco/', 'https://maldita.es/malditobulo/2020/02/28/jaki-bulo-coronavirus-totana/', 'https://maldita.es/malditobulo/2020/02/28/hombre-comprando-con-una-mascara-antigas-supermercado-no-ha-sido-tomada-en-lugo-ni-espana-fue-en-austria-coronavirus/', 'https://maldita.es/malditobulo/2020/02/28/cifras-musulmanes/', 'https://maldita.es/malditobulo/2020/02/28/quaden-bayles-enanismo-acondroplasia-disneyland-estafa-australia/', 'https://maldita.es/malditobulo/2020/02/27/vitamina-c-omega-coronavirus/', 'https://maldita.es/malditobulo/2020/02/27/coronavirus-alcala-de-henares/', 'https://maldita.es/malditobulo/2020/02/27/no-el-diario-el-pais-no-ha-publicado-que-exista-un

In [40]:
lista_articulos

[{'Title': 'No, este vídeo de unos chicos negros abriendo y bebiendo refrescos en un supermercado no es de España',
  'Publication_date': '28/02/2020',
  'Content': 'Nos habéis preguntado por un vídeo que está circulando en redes y que, además, el presidente Provincial de VOX en Girona, Alberto Tarradas Panenque, ha difundido en su perfil de Twitter en el que aseguran que los chicos negros que aparecen bebiendo refrescos de un supermercado son "los \'nuevos catalanes\', totalmente integrados y trabajando para pagar las pensiones de nuestros mayores..." y les señalan de "pagadores de pensiones". Es un bulo: el vídeo no se ha grabado en un supermercado de España.La primera referencia que hemos encontrado al vídeo es esta publicación del 16 de febrero de la página de Facebook marfileña Ivoiriens D\'abord, que titula las imágenes con el siguiente pie: "ven a ver lo que pasa dentro de nuestros supermercados", en francés.El francés es la lengua oficial de Costa de Marfil, y en un momento del

In [41]:
#Guardamos los datos:
from datetime import date

date = date.today().strftime("%d-%m-%Y")

filename = 'Maldita_FN_Pag-' + str(pagina) + '_'+ date
print(filename)

outfile = open(filename,'wb')
pickle.dump(lista_articulos, outfile)  
    
outfile.close()

Maldita_FN_Pag-34_14-06-2020


In [42]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': 'No, este vídeo de unos chicos negros abriendo y bebiendo refrescos en un supermercado no es de España',
  'Publication_date': '28/02/2020',
  'Content': 'Nos habéis preguntado por un vídeo que está circulando en redes y que, además, el presidente Provincial de VOX en Girona, Alberto Tarradas Panenque, ha difundido en su perfil de Twitter en el que aseguran que los chicos negros que aparecen bebiendo refrescos de un supermercado son "los \'nuevos catalanes\', totalmente integrados y trabajando para pagar las pensiones de nuestros mayores..." y les señalan de "pagadores de pensiones". Es un bulo: el vídeo no se ha grabado en un supermercado de España.La primera referencia que hemos encontrado al vídeo es esta publicación del 16 de febrero de la página de Facebook marfileña Ivoiriens D\'abord, que titula las imágenes con el siguiente pie: "ven a ver lo que pasa dentro de nuestros supermercados", en francés.El francés es la lengua oficial de Costa de Marfil, y en un momento del